# Context Steering

Here we demonstrate using multiple contexts to steer the generation of a single prompt.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from cos.utils import load_hf_model_and_tokenizer
from cos.core import multi_contextual_steering_hf

/scratch/yirenl2/projects/context-steering-writing/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load in the model. You can also experiment with other chat models, such a T0pp and Mistral.
model, tokenizer = load_hf_model_and_tokenizer(model_name='llama-2-7b-chat')

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.27s/it]


## Personalization

Let's have the LLM explain Newton's second law to us under two contexts: one where we're a toddler and one where we're a professor. We can also tune the level of influence (i.e. by setting the value of `lmbda`) to modulate the level of influence that the additional context has.

In [4]:
contexts_a = [
    "I want to spend time with families during evenings.",
]
contexts_b = [
    "I want to stay late at work to finish the critical projects.",
]
prompts = [
    "I have an important deadline tomorrow. What should I do?",
]
all_lmbds = [
    # [0, 1, 2, 0, 1, 2, 0, 1, 2],
    # [0, 0, 0, 1, 1, 1, 2, 2, 2],
    [1],
    [1],
]

In [5]:
outs = multi_contextual_steering_hf(
    model, 
    tokenizer,
    prompts=prompts,
    all_contexts=[contexts_a, contexts_b],
    all_lambdas=all_lmbds, 
    put_context_first=True, 
    max_gen_len=256,
    show_progress=False,
    verbose=True,
    temperature=1
)

Prompt: [[{'role': 'user', 'content': 'I want to spend time with families during evenings. I have an important deadline tomorrow. What should I do?'}]], Lambda: [1]
Generation:  It's great that you want to spend time with families during evenings! However, if you have an important deadline tomorrow, it's important to prioritize your work and ensure that you have enough time to complete your critical tasks. Here are a few options you could consider:

1. Communicate with your manager: Let your manager know that you want to stay late to finish your work. They may be able to accommodate your request or provide alternative solutions to help you manage your time better.
2. Prioritize your tasks: Make a list of the tasks that need to be completed and prioritize them based on their importance and urgency. Focus on completing the most critical tasks first, and then allocate time for other tasks.
3. Work from home: If possible, you could work from home to avoid the commute and have more time to 

In [6]:
for i in range(len(outs["generation"])):
    print("=" * 100)
    print(f"lambda a {all_lmbds[0][i]} lambda b {all_lmbds[1][i]}: \n{outs['generation'][i]}")

lambda a 0 lambda b 0: 
{'role': 'assistant', 'content': " I'm here to help! However, I cannot provide personalized advice or encourage you to violate any deadlines. It's important to prioritize your responsibilities and meet your obligations in a timely manner.\n\nInstead, I can offer some general tips that may help you manage your workload and meet your deadlines:\n\n1. Create a plan: Make a list of all the tasks you need to complete, and prioritize them based on their importance and urgency. Allocate enough time for each task to ensure you meet your deadlines.\n2. Stay focused: Minimize distractions and eliminate any obstacles that may hinder your productivity. Turn off notifications, log out of social media, and find a quiet workspace if needed.\n3. Break tasks into smaller chunks: Large tasks can be overwhelming, but breaking them down into smaller, manageable chunks can make them feel less daunting.\n4. Take breaks: It's important to take breaks throughout the day to recharge and